In [42]:
import torch

In [47]:
a = torch.zeros((2, 288)).type(torch.int64)

for row in a:
    print(row)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [68]:
a.unsqueeze(0).shape

torch.Size([1, 2, 288])

In [62]:
import datetime
import math
import sys, os

import numpy as np
import sentencepiece as spm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import BatchEncoding, BertForMaskedLM


SEQUENCE_LENGTH = 288


class BPEBertTokeniser:
    out_keys = [
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "special_tokens_mask",
    ]

    def __init__(self, lang):
        self.lang = lang
        self.pad_token_id = 3

    def _process_id(self, input_ids):
        """
        Called inside the model.
        """

        # src_i: EOS, # trg_i: BOS, #trg_o: EOS
        # input_ids = torch.tensor(input_ids) # 288, padded

        token_type_ids = torch.zeros(input_ids.shape)

        attention_mask = torch.where(input_ids == self.pad_token_id, 0, 1)

        special_tokens_mask = torch.where(input_ids < 4, 1, 0)

        return {
            "input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_mask,
            "special_tokens_mask": special_tokens_mask,
        }

    def encode(self, row):
        if len(row.shape) > 1:
            return self.encode_batch(row)
        return self._process_id(row)

    def encode_batch(self, rows):
        ids = list(map(lambda row: self._process_id(row), rows))
        return {key: torch.vstack([example[key] for example in ids]) for key in BPEBertTokeniser.out_keys}
    

    def __call__(self, inputs, **_kwargs):
        return self.encode(inputs)

    def __len__(self):
        return len(self.model)



DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


class BPEBertEmbedder:
    def __init__(self, lang: str, model_file: str):
        # Init the model from the pretrained weights
        self.model = BertForMaskedLM.from_pretrained(
            model_file, output_hidden_states=True
        ).to(DEVICE)
        self.model.eval()

        # Init the BPE tokeniser (padded to length 288, vocab 16384)
        self.tokeniser = BPEBertTokeniser(lang)
        pass

    def embed(self, sent: list[int]):
        tokens = self.tokeniser(
            sent
        )  # input_ids, token_type_mask, attention_mask, special_tokens_mask
        with torch.no_grad():
            input_ids = tokens["input_ids"].to(DEVICE)
            token_type_ids = tokens["token_type_ids"].to(DEVICE)
            attention_mask = tokens["attention_mask"].to(DEVICE)
            # Perform inference
            output = self.model(input_ids, token_type_ids, attention_mask)
            hidden_states = output.hidden_states

            # Permutate and obtain hidden states
            token_embeddings = torch.stack(hidden_states, dim=0)  # 13, batch, 288, 256

            print(token_embeddings.shape)
            token_embeddings = token_embeddings.permute(1, 2, 0, 3)  # (batch, 288, n_layers, 256)

            # Take last 4 layers
            token_embeddings = token_embeddings[:, :, -4:, :]

            # Take sum of last 4 layers
            token_embeddings = token_embeddings.sum(axis=2)  # (288, 256)

            # Matrix of embeddings of dim 256, one per word
            return token_embeddings

en_embedder = BPEBertEmbedder(
    lang="en",
    model_file='../../models/bert-embed/BPE-bert-en',
    # spm_file='../../tokenisation/sentencepiece_custom/en.model',
)

zh_embedder = BPEBertEmbedder(
    lang="zh",
    model_file='../../models/bert-embed/BPE-bert-zh',
    # spm_file='../../tokenisation/sentencepiece_custom/zh.model',
)


In [64]:
en_embedder.embed(a).shape

torch.Size([13, 2, 288, 256])


torch.Size([2, 288, 256])

In [10]:
with open('../../tokenisation/data/iwslt2017-en-zh-test.en', 'r') as f:
    sent = f.readline()

e = en_embedder.embed(sent)

In [11]:
em, p = e

In [12]:
em.shape

torch.Size([288, 256])

In [53]:
from e2e import init_model

In [54]:
model = init_model(16384, 16384, '../../models/transformer-scratch/230k-rows_15epoch-btch-90-embed-288.tar')

In [55]:
model.src_embedding

Embedding(16384, 256)

In [56]:
model.positional_encoder

PositionalEncoder()

In [66]:
model.positional_encoder(model.src_embedding(torch.zeros((90,288)).type(torch.int32))).shape

torch.Size([90, 288, 256])

In [16]:
import sentencepiece as spm

In [17]:
tokeniser = spm.SentencePieceProcessor('../../tokenisation/sentencepiece_custom/en.model')

In [19]:
def get_ids():
    ids = tokeniser.encode('I want to go to school.')
    ids += [3] * (288 - len(ids))
    return ids

In [32]:
i = [[get_ids()] * 256] * 10

In [37]:
torch.tensor(i).permute((0, 2, 1)).shape

torch.Size([10, 288, 256])

In [41]:
(torch.tensor(i).permute((0, 2, 1)) == 3).unsqueeze(1)[:, :, :, 0].shape

torch.Size([10, 1, 288])